# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
#import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [9]:
df_test.head(5)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני..."
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [10]:
df_train['gender'].value_counts()

m    575
f    178
Name: gender, dtype: int64

In [11]:
df_train.describe()

,story,gender
count,753,753
unique,749,2
top,"כזוג מאורס טרי סגרנו תאריך, אולם אירועים, ספקי...",m
freq,2,575


# Data Pre-Processing


## Tokenization

In [12]:
# Function that clean the story and split him to tokens
def makeTokenization(story):
    
    story = story.strip(',:;{} ')
    story = story.replace("'", "").replace(".", "").replace("\\", "").replace(",", "").replace(";", "").replace("!","").replace("?","").replace("(","").replace(")","").replace("*", "").replace("/", "")
    story = re.sub(r'\d+', '', story)
    story = re.sub(r'[a-z]+|[A-Z]+', '', story)
    story = story.split(' ')
        
    return story

In [13]:
# Testing the tokenization function
a = makeTokenization("טסט טסט טסט123123 ;;;")
print(a)

['טסט', 'טסט', 'טסט']


In [14]:
# Make the gender col to numeric values
df_train['gender'] = df_train['gender'].map({'f':0, 'm':1})

# Vectorization

In [15]:
vec = TfidfVectorizer(tokenizer=makeTokenization, max_features=1500, max_df=0.8)
X_train = vec.fit_transform(df_train.story)
y_train = df_train.gender

# Normalize
X_train = preprocessing.normalize(X_train, norm='l2')

# Model Selection

## Cross Validation & GridSearchCV

In [16]:
# finding best params using gridsearch(cross validation)

# create a list of model names
model_names = [
    'KNN',
     'DT',
#     'MLP',
    'Perceptron',
    'SVM'
]

f1_acc_li = []
bestScore = 0
bestModel = None

# --------------------------------------------------------------------------------- #

def train_model(model, X_train, y_train):
    if(model == 'KNN'):
        clf = KNeighborsClassifier()
        clf.fit(X_train, y_train)
        
    if(model == 'DT'):
        clf = DecisionTreeClassifier()
        clf.fit(X_train, y_train)
        
#     if(model == 'MLP'):
#         clf = MLPClassifier()
#         clf.fit(X_train, y_train)
        
    if(model == 'Perceptron'):
        clf = Perceptron()
        clf.fit(X_train, y_train)
        
    if(model == 'SVM'):
        clf = SGDClassifier()
        clf.fit(X_train, y_train)
        
    return clf

def grid_search_model(model, clf, X_train, y_train):
    
    if(model == 'KNN'):
        parameters = {'metric':['euclidean', 'manhattan', 'minkowski'],
                      'n_neighbors':[11, 20, 25, 30],
                      'weights':['uniform', 'distance']}
        
    if(model == 'DT'):
        parameters = {'criterion':['gini', 'entropy'],
                      'max_depth':[ 4, 5, 6, 7, 8],
                      'min_samples_leaf':[ 5, 6, 7, 8, 9], 
                      'min_samples_split':[10, 15, 20, 25]}
        
#     if(model == 'MLP'):
#             parameters = {'alpha':[0.0001, 0.0013, 0.0133], 
#                           'solver':['lbfgs', 'sgd', 'adam']}
        
    if(model == 'Perceptron'):
            parameters = {'penalty':['l2', 'l1', 'elasticnet'],
                          'alpha':[0.0001, 0.0002, 0.0003]}
        
    if(model == 'SVM'):
        parameters = {'loss':['hinge', 'log', 'modified_huber','squared_hinge', 'perceptron'], 
                      'penalty':['l2', 'l1', 'elasticnet'], 
                      'alpha':[0.0001, 0.002, 0.0003]}
    
    GS_model = GridSearchCV(estimator=clf, param_grid=parameters, cv=5)
    GS_model.fit(X_train, y_train)
    return GS_model

for model in range(len(model_names)):
    clf = train_model(model_names[model], X_train, y_train)
    GS_model = grid_search_model(model_names[model], clf, X_train, y_train)
    print(GS_model.best_params_)
    
    model_cv_score = cross_val_score(GS_model.best_estimator_, X_train, y_train, scoring=metrics.make_scorer(f1_score, average='macro'), cv=5)
    print(f'{model_names[model]} Cross Validation Scores: {model_cv_score}')
    print(f'{model_names[model]} Mean Accuracy: {model_cv_score.mean():.4f}')
    
    if(model_cv_score.mean() > bestScore):
        bestScore = model_cv_score.mean()
        bestModel = model_names[model]
        
    print()

print(f"The best model is: {bestModel}\n")
print(f"The best f1 score is {bestScore}\n")

# Train the trainSet with the best algoritem
bestClf = train_model(bestModel, X_train, y_train)
GS_best_model = grid_search_model(bestModel, bestClf, X_train, y_train)
bestModel_cv_score = cross_val_score(GS_best_model.best_estimator_, X_train, y_train, scoring=metrics.make_scorer(f1_score, average='macro'), cv=5)
print(f'{bestModel} Cross Validation Scores: {bestModel_cv_score}')
print(f'{bestModel} Mean Accuracy: {bestModel_cv_score.mean():.4f}')

{'metric': 'manhattan', 'n_neighbors': 20, 'weights': 'distance'}
KNN Cross Validation Scores: [0.43018868 0.46098929 0.46098929 0.43396226 0.49131641]
KNN Mean Accuracy: 0.4555

{'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 6, 'min_samples_split': 20}
DT Cross Validation Scores: [0.58423029 0.52114165 0.58714163 0.51215337 0.56350565]
DT Mean Accuracy: 0.5536

{'alpha': 0.0003, 'penalty': 'elasticnet'}
Perceptron Cross Validation Scores: [0.56857143 0.69391892 0.70881543 0.70826275 0.69555323]
Perceptron Mean Accuracy: 0.6750

{'alpha': 0.0001, 'loss': 'log', 'penalty': 'l1'}
SVM Cross Validation Scores: [0.6891147  0.66818216 0.58714163 0.66124661 0.69262295]
SVM Mean Accuracy: 0.6597

The best model is: Perceptron

The best f1 score is 0.6750243516569394

Perceptron Cross Validation Scores: [0.56857143 0.69391892 0.70881543 0.70826275 0.69555323]
Perceptron Mean Accuracy: 0.6750


# test

In [17]:
X_test = vec.transform(df_test.story)
# Normalize
X_test = preprocessing.normalize(X_test, norm='l2')

# Model Evaluation

In [18]:
y_pred = clf.predict(X_test)

df_test['gender'] = y_pred
df_test['gender'] = df_test['gender'].replace({0:'f', 1:'m'})
print(df_test.head(5))
print()
print(df_test.tail(5))
print()
print(f"Males: {np.count_nonzero(y_pred)}")
print(f"Females: {y_pred.shape[0] - np.count_nonzero(y_pred)}")
print()
print(y_pred)

   test_example_id                                              story gender
0                0  כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...      m
1                1  הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת "...      m
2                2  אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...      m
3                3  רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...      m
4                4  אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...      f

     test_example_id                                              story gender
318              318  בשנה האחרונה הרגשתי די תקוע בעבודה, השגרה הפכה...      m
319              319  אני ואילן חברים טובים מזה 20 שנה תמיד חלמנו לפ...      m
320              320  מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...      m
321              321  לפני מספר חודשים, בשיא התחלואה של הגל השני, עמ...      m
322              322  היום בו דיווחתי על גניבה של האוטו שלי. בוקר אח...      m

Males: 252
Females: 71

[1 1 1 1 0 1 1 0 1 0 0 1 1 1 0 1 1 1 1

# END

### Tests

### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [19]:
#df_predicted.to_csv('classification_results.csv',index=False)